### This file creates a dict with each word across all the con files and takes its count

In [28]:
import pandas as pd
import numpy as np
import glob
import re
from nltk.corpus import stopwords
import string
import operator

In [2]:
d = dict()
stop = stopwords.words('english') + list(string.punctuation)

In [26]:
for filename in glob.glob('samples/*.con'):
#     print("****************************************************************")
#     print(filename)
#     print("****************************************************************")
    with open(filename) as f:
        for line in f:
            if not line:
                pass
            else:
                if len(line.split("\"")) == 5:
                    temp = line.split("\"")[1]
                    x = temp.translate(None, string.punctuation)
                    temp = x.strip()
        #             temp_arr = [i for i in temp.split() if i not in stop]
                    if temp in d:
                        d[temp] += 1

                    else:
                        d[temp] = 0
                        d[temp] += 1
            


In [107]:
df = pd.DataFrame.from_dict(d, orient='index')
df.columns=['Values']

In [108]:
df = df.sort_values(by=['Values'], ascending=False)

In [109]:
df.head()

,Values
type num diabetes,642
type num diabetes mellitus,615
treatment,534
the study,496
insulin,469


In [110]:
df.drop(df.head(4).index, inplace=True)

In [111]:
df.head()

,Values
insulin,469
metformin,448
type 2 diabetes mellitus,356
diabetes,324
type 2 diabetes,316
